# How Facebook can save meaningful conversation

Any medium have one clear objective that it must support effective and meaningful conversation. Social media is not exception for it. But recently social media is imfamous for chaos instead of communication. In my personal opinion most of time trolling leads to this chaos. 

Advancement in NLP and machine learning enhance ablities of machine to understand human languages with it's nuances.       

There are multiple ways we can use to identify/classify troll comments on any random post. Following code will show simple way to implement troll classifier. With help of it we can define importance of comments according to it's meaning and sort comments order in such a way that more meaningful comments get more visibility.

Instead of creating classifier from scratch, we can make use of transfer learning. 
By defination storing knowledge gaining while solving one problem and applying it one different but related problem is known as transfer learning.


For language understanding we are using BERT. BERT is pre-trained contextual language representation developed by Google. Hence instead of start with dataset generation to model generation/validation we directly use language model to understand nuance and identify trolling. 

Here for language understanding we are using bert-as-a-service implementation.

In [ ]:
!pip install bert-serving-server  # server
!pip install bert-serving-client  # client

Externally (i.e. in separate console) we start BERT service with BERT language model

In [ ]:
bert-serving-start -model_dir /home/pritam/bert-models/uncased_L-12_H-768_A-12 num_worker=4

Following dependencies need to declare

In [1]:
import numpy as np
from bert_serving.client import BertClient


Encapsulate comparison of post with single comment in class Comment.

In [19]:
class Comment:
    def __init__(self,post,comment):
        super()
        self.bc = BertClient()
        self.post = post
        self.comment = comment
        self.post_embed = bc.encode([self.post])[0]
        self.comment_embed = bc.encode([self.comment])[0] 
        
    def get_similarity(self):
        return np.dot(self.post_embed, self.comment_embed) / (np.linalg.norm(self.post_embed) * np.linalg.norm(self.comment_embed))


Following is one simple example contains text of social media post with comments on that post.

In [66]:
post = 'Thousands say their final goodbyes to Bradley Lowery.'
comments = ['such tragic incident can avoided','Rest in peace','World will never forget you','can\'t people see and walk','so what can I do','There is solution for such incident, one must be attentive on road','rediculous','Lol']
#comment1 = Comment(post,comment)

convert plain texts in necessary object list.

In [67]:
def get_Comment_list(post,comments):
    comment_list = []
    for comment in comments:
        comment_obj = Comment(post,comment)
        comment_list.append(comment_obj)
    return comment_list    

In [68]:
comment_list = get_Comment_list(post,comments)

Sort comments according to it's relevance score.

In [69]:
sorted_coment_list = sorted(comment_list,key=lambda x : x.get_similarity(), reverse=True)

Now you can see most relevant comments according to algorithm comes first. Let us see quality of classification.

In [70]:
for comment in sorted_coment_list:
    print(comment.comment)

World will never forget you
Rest in peace
such tragic incident can avoided
can't people see and walk
There is solution for such incident, one must be attentive on road
Lol
so what can I do
rediculous


This is pretty simple implementation for troll comment classification. Going further we can use more complex mechanism. In this approach we are treating similarity and relevance in same way. Sometimes relevance and similarities may have different meanings.

Interesting Readings:
https://medium.com/analytics-vidhya/semantic-similarity-in-sentences-and-bert-e8d34f5a4677
http://nlp.town/blog/sentence-similarity/